# Azure AI Agent service - Bing integration

<img src="https://learn.microsoft.com/en-us/azure/ai-services/agents/media/agent-service-the-glue.png" width=800>

> https://learn.microsoft.com/en-us/azure/ai-services/agents/

In [1]:
#%pip install azure-ai-projects

In [2]:
import datetime
import os
import sys

from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.projects.models import BingGroundingTool
from dotenv import load_dotenv

In [3]:
load_dotenv("azure.env")

True

In [4]:
sys.version

'3.10.14 (main, May  6 2024, 19:42:50) [GCC 11.2.0]'

In [5]:
print(f"Today is {datetime.datetime.today().strftime('%d-%b-%Y %H:%M:%S')}")

Today is 20-May-2025 13:06:27


## 1. Grounding Bing Search

In [6]:
name = "Web search agent"
model = "gpt-4o"

instructions = "You are an AI Agent that can do some web search using Bing"
connection_name = "groundingbingsearch"

### Project client

In [7]:
# Create an Azure AI Client from a connection string, copied from your Azure AI Foundry project.
# It should be in the format "<HostName>;<AzureSubscriptionId>;<ResourceGroup>;<HubName>"
# Customers need to login to Azure subscription via Azure CLI and set the environment variables

In [8]:
project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str=os.getenv("PROJECT_CONNECTION_STRING"),
)

In [9]:
bing_connection = project_client.connections.get(connection_name=connection_name)
conn_id = bing_connection.id

# Initialize agent bing tool and add the connection id
bing = BingGroundingTool(connection_id=conn_id)

In [10]:
# Create agent with the bing tool and process assistant run
agent = project_client.agents.create_agent(
    model=model,
    name=name,
    instructions=instructions,
    tools=bing.definitions,
    headers={"x-ms-enable-preview": "true"}
)

print(f"Created agent, ID: {agent.id}")

Created agent, ID: asst_nWbdIExnQumrKa5nsuDeteib


In [11]:
# Create thread for communication
thread = project_client.agents.create_thread()
print(f"Created thread, ID: {thread.id}")

Created thread, ID: thread_360vcZDfpbRCIhCpKTRpN2KQ


In [12]:
query = "What are the latest annoucements of Microsoft Build 2025?"

In [13]:
# Create message to thread
message = project_client.agents.create_message(
    thread_id=thread.id,
    role="user",
    content=query,
)

print(f"Created message, ID: {message.id}")

Created message, ID: msg_14SYbMS4HgRfqYtjlbGuKvf5


In [14]:
# Create and process agent run in thread with tools
run = project_client.agents.create_and_process_run(thread_id=thread.id, agent_id=agent.id)
print(f"Run finished with status: {run.status}")

run_steps = project_client.agents.list_run_steps(run_id=run.id, thread_id=thread.id)
run_steps_data = run_steps['data']
print(f"Last run step detail: {run_steps_data}")

# Fetch and log all messages
messages = project_client.agents.list_messages(thread_id=thread.id)
print(f"Messages: {messages}")

Run finished with status: completed
Last run step detail: [{'id': 'step_202bEP0Lvko37iFx7xMdTTTx', 'object': 'thread.run.step', 'created_at': 1747746397, 'run_id': 'run_J18Pr0YmfhfRBKcfFr1nuy6h', 'assistant_id': 'asst_nWbdIExnQumrKa5nsuDeteib', 'thread_id': 'thread_360vcZDfpbRCIhCpKTRpN2KQ', 'type': 'message_creation', 'status': 'completed', 'cancelled_at': None, 'completed_at': 1747746398, 'expires_at': None, 'failed_at': None, 'last_error': None, 'step_details': {'type': 'message_creation', 'message_creation': {'message_id': 'msg_w7rEVKEisIe852xLpmgPRS2i'}}, 'usage': {'prompt_tokens': 1677, 'completion_tokens': 85, 'total_tokens': 1762, 'prompt_token_details': {'cached_tokens': 0}}}, {'id': 'step_Y2T4v3Pn3ejKGYijWTyZ7Pgv', 'object': 'thread.run.step', 'created_at': 1747746395, 'run_id': 'run_J18Pr0YmfhfRBKcfFr1nuy6h', 'assistant_id': 'asst_nWbdIExnQumrKa5nsuDeteib', 'thread_id': 'thread_360vcZDfpbRCIhCpKTRpN2KQ', 'type': 'tool_calls', 'status': 'completed', 'cancelled_at': None, 'com

In [15]:
print("\033[1;31;34m")
print(messages.data[0].content[0].text.value)


Microsoft Build 2025 highlighted advancements in AI, including “agentic AI” tools designed to revolutionize productivity across apps and platforms. Major announcements included enhancements to GitHub Copilot's autonomous coding capabilities, Windows AI features, updated Azure development tools, and the debut of AI-based discovery platforms for business and scientific innovation【3:0†source】【3:7†source】【3:11†source】.


In [16]:
query = "What is the current stock value for Microsoft?"

In [17]:
# Create message to thread
message = project_client.agents.create_message(
    thread_id=thread.id,
    role="user",
    content=query,
)
print(f"Created message, ID: {message.id}")

Created message, ID: msg_NpBXvE0Gq8TDzuq3dHfgzDtH


In [18]:
# Create and process agent run in thread with tools
run = project_client.agents.create_and_process_run(thread_id=thread.id, agent_id=agent.id)
print(f"Run finished with status: {run.status}")

run_steps = project_client.agents.list_run_steps(run_id=run.id, thread_id=thread.id)
run_steps_data = run_steps['data']
print(f"Last run step detail: {run_steps_data}")

Run finished with status: completed
Last run step detail: [{'id': 'step_OfVPTptjkOu9AJkYPqSElS1s', 'object': 'thread.run.step', 'created_at': 1747746406, 'run_id': 'run_ayur8jmpbqFKUy7D4XNj9X6F', 'assistant_id': 'asst_nWbdIExnQumrKa5nsuDeteib', 'thread_id': 'thread_360vcZDfpbRCIhCpKTRpN2KQ', 'type': 'message_creation', 'status': 'completed', 'cancelled_at': None, 'completed_at': 1747746407, 'expires_at': None, 'failed_at': None, 'last_error': None, 'step_details': {'type': 'message_creation', 'message_creation': {'message_id': 'msg_1xfA6iPvWth7qvXrhBYZVWul'}}, 'usage': {'prompt_tokens': 1408, 'completion_tokens': 42, 'total_tokens': 1450, 'prompt_token_details': {'cached_tokens': 0}}}, {'id': 'step_VN5Xdf2Yca0zDNENkNkB3o1u', 'object': 'thread.run.step', 'created_at': 1747746404, 'run_id': 'run_ayur8jmpbqFKUy7D4XNj9X6F', 'assistant_id': 'asst_nWbdIExnQumrKa5nsuDeteib', 'thread_id': 'thread_360vcZDfpbRCIhCpKTRpN2KQ', 'type': 'tool_calls', 'status': 'completed', 'cancelled_at': None, 'com

In [19]:
print("\033[1;31;34m")
messages = project_client.agents.list_messages(thread_id=thread.id)
print(messages.data[0].content[0].text.value)


The current stock value for Microsoft (MSFT) is $458.87 as of the latest closing session. Please note that stock prices can fluctuate during trading hours【7:9†source】.


In [20]:
query = "What are the weather informations for Paris? Use emojis and be funny"

In [21]:
# Create message to thread
message = project_client.agents.create_message(
    thread_id=thread.id,
    role="user",
    content=query,
)
print(f"Created message, ID: {message.id}")

Created message, ID: msg_TMBC71nXdjROxFFuAbasg4f0


In [22]:
# Create and process agent run in thread with tools
run = project_client.agents.create_and_process_run(thread_id=thread.id, agent_id=agent.id)
print(f"Run finished with status: {run.status}")

run_steps = project_client.agents.list_run_steps(run_id=run.id, thread_id=thread.id)
run_steps_data = run_steps['data']
print(f"Last run step detail: {run_steps_data}")

Run finished with status: completed
Last run step detail: [{'id': 'step_WQjWl8lc6ZD8wqPvrMTARaFv', 'object': 'thread.run.step', 'created_at': 1747746414, 'run_id': 'run_n7Yu7kJXlUu2hoKmyBzyor9h', 'assistant_id': 'asst_nWbdIExnQumrKa5nsuDeteib', 'thread_id': 'thread_360vcZDfpbRCIhCpKTRpN2KQ', 'type': 'message_creation', 'status': 'completed', 'cancelled_at': None, 'completed_at': 1747746416, 'expires_at': None, 'failed_at': None, 'last_error': None, 'step_details': {'type': 'message_creation', 'message_creation': {'message_id': 'msg_q4YjDar8IdvauqNGrAZXbeA8'}}, 'usage': {'prompt_tokens': 1552, 'completion_tokens': 79, 'total_tokens': 1631, 'prompt_token_details': {'cached_tokens': 0}}}, {'id': 'step_vysZHTk5SrHNPy7WuCMKqkdD', 'object': 'thread.run.step', 'created_at': 1747746412, 'run_id': 'run_n7Yu7kJXlUu2hoKmyBzyor9h', 'assistant_id': 'asst_nWbdIExnQumrKa5nsuDeteib', 'thread_id': 'thread_360vcZDfpbRCIhCpKTRpN2KQ', 'type': 'tool_calls', 'status': 'completed', 'cancelled_at': None, 'com

In [23]:
print("\033[1;31;34m")
messages = project_client.agents.list_messages(thread_id=thread.id)
print(messages.data[0].content[0].text.value)


Today in Paris, it's 🌞 sunny with scattered clouds ☁️. Temperatures range from 12°C 🍂 in the morning to a comfy 24°C 🌷 in the afternoon—perfect for café-hopping! No rain ☔, but keep your sunglasses 😎 handy. Paris is glowing like the Eiffel Tower at night 🌟! 【11:2†source】


### Post processing

In [24]:
agents = project_client.agents.list_agents()

for i in range(len(agents.data)):
    if agents.data[i].name == name:
        print(f"Deleting agent {agents.data[i].id}")
        project_client.agents.delete_agent(agents.data[i].id)

Deleting agent asst_nWbdIExnQumrKa5nsuDeteib
